# Performance Baselines

In [1]:
import watermark
import pandas as pd
import numpy as np
import datetime
import random
import copy
import os
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import mean_squared_error
import ray
from tqdm import tqdm

%reload_ext watermark
%watermark -p xgboost,pandas,numpy,scikit-learn
!python --version

xgboost     : 1.5.1
pandas      : 1.3.4
numpy       : 1.21.3
scikit-learn: 0.0

Python 3.8.12


## Dataset

In [2]:
# Read ratings df
path = "data_small"
use_large = False
if (use_large):
    path = "data_large"
    ratings = pd.read_csv('./' + path + '/ratings.csv')[:1000000]
else:
    ratings = pd.read_csv('./' + path + '/ratings.csv')

movies = pd.read_csv('./' + path + '/movies.csv')

print(ratings)

        userId  movieId  rating   timestamp
0            1        1     4.0   964982703
1            1        3     4.0   964981247
2            1        6     4.0   964982224
3            1       47     5.0   964983815
4            1       50     5.0   964982931
...        ...      ...     ...         ...
100831     610   166534     4.0  1493848402
100832     610   168248     5.0  1493850091
100833     610   168250     5.0  1494273047
100834     610   168252     5.0  1493846352
100835     610   170875     3.0  1493846415

[100836 rows x 4 columns]


In [3]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [4]:
genres_set = set()
for string in tqdm(movies.genres):
    genres_set.update(string.split('|'))
#genres.remove('(no genres listed)')
genres = sorted(genres_set)
print(genres)

100%|██████████| 9742/9742 [00:00<00:00, 1415047.43it/s]

['(no genres listed)', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']


In [5]:
for genre in genres:
    ratings[genre] = False

for movie in tqdm(movies.movieId):
    for genre in movies.genres[movies.loc[movies.movieId == movie].index[0]].split('|'):
        ratings.loc[ratings.movieId == movie, [genre]] = True

100%|██████████| 9742/9742 [00:20<00:00, 474.07it/s]


In [6]:
# Delete movies with fewer than 25 ratings
data = ratings.pivot(index = "userId", columns = "movieId", values = "rating")
print(ratings.shape)
to_del = list()
counts = data.count(axis = 0, numeric_only = True)
for i in data.columns.values:
    if (counts[i] < 25):
        to_del.append(i)
ratings = ratings[~(ratings.movieId.isin(to_del))]
print(ratings.shape)
del data

(100836, 24)
(62518, 24)


In [7]:
@ray.remote
def insufficient_ratings(dataset, user):
    user_ratings = sum(dataset.userId == user)
    if (user_ratings < 20):
        return user
    return -1

In [8]:
# Removes users who no longer have 20 ratings

def to_iterator(obj_ids):
    while obj_ids:
        done, obj_ids = ray.wait(obj_ids)
        yield ray.get(done[0])

dat = ray.put(ratings)
obj_ids = [insufficient_ratings.remote(dat, user) for user in ratings.userId.unique()]
results = []
for x in tqdm(to_iterator(obj_ids), total = len(obj_ids)):
    if x > -1:
        results.append(x)

ratings = ratings.loc[~(ratings.userId.isin(results))]

del dat
        
print("There were", len(results), "users with fewer than 20 ratings. They have been removed from the dataset.")

100%|██████████| 610/610 [00:01<00:00, 483.27it/s]

There were 61 users with fewer than 20 ratings. They have been removed from the dataset.


In [9]:
# Note that GroupShuffleSplit allows us to have distinct users in the training, validation, and test sets

# Split into 80-20 training - testing split
train_inds, test_inds = next(GroupShuffleSplit(test_size=.20, n_splits=2, random_state = 42).split(ratings, groups=ratings.userId))

train_val = ratings.iloc[train_inds].copy()
test = ratings.iloc[test_inds].copy()

# Split into 80-20 training - validation split
train_inds, val_inds = next(GroupShuffleSplit(test_size=.20, n_splits=2, random_state = 42).split(train_val, groups=train_val.userId))

training = train_val.iloc[train_inds].copy()
validation = train_val.iloc[val_inds].copy()

print('Train/Valid/Test sizes:', training.shape[0], validation.shape[0], test.shape[0])

Train/Valid/Test sizes: 38700 10195 12724


In [10]:
# As it is, this will take the 10 most recent ratings for a user to use for prediction
# Commented out sections allow random ratings to be taken

@ray.remote
def prepare_helper(user, dataset):
    #random.seed(1)
    user_ratings = dataset.loc[dataset.userId == user]
                               
    # Randomly select 10 movies to remove
    #selected_movies = random.sample(range(0, user_ratings.shape[0]), 10)
    #selected_movies = random.choices(list(user_ratings.movieId), k=10)
                               
    # This will select the 10 most recent ratings for this user
    selected_movies = user_ratings.sort_values(by = "timestamp", ascending = False).movieId.values[0:10]
    
    # Add the removed movies to the removed dataframe
    removed_movies = user_ratings[user_ratings.movieId.isin(selected_movies)]
    
    return removed_movies

In [11]:
# This function takes in either the validation or training set
# Removes the most recent 10 ratings for each user, and returns them as removed_movies
# Returns the original dataset with the ratings to predict removed as new_movies
def prepare_data(dataset):
    # For validation and test data, remove 10 rated movies to be predicted
    removed_movies = pd.DataFrame(columns = dataset.columns)
    # Iterate over each user
    
    def to_iterator(obj_ids):
        while obj_ids:
            done, obj_ids = ray.wait(obj_ids)
            yield ray.get(done[0])
    dat = ray.put(dataset)
    obj_ids = [prepare_helper.remote(user, dat) for user in dataset["userId"].unique()]
    results = []
    for x in tqdm(to_iterator(obj_ids), total = len(obj_ids)):
        results.append(x)
    
    # Combine removed movies:
    for result in tqdm(results):
        removed_movies = pd.concat([removed_movies, result])
        
    # Remove all of the user-movie combinations in removed_moviews from dataset
    new_movies = pd.merge(dataset, removed_movies, indicator=True, how='outer').query('_merge=="left_only"').drop('_merge', axis=1)
    del dat
    
    return removed_movies, new_movies

In [12]:
# Get the proper splits for the validation and testing sets
if (os.path.isfile('./' + path + '/validation_removed_tmp.csv') and os.path.isfile('./' + path + '/validation_new.csv')):
    print("Validation set already exists" )
    validation_removed = pd.read_csv('./' + path + '/validation_removed_tmp.csv')
    validation_new = pd.read_csv('./' + path + '/validation_new.csv')
else:
    start = datetime.datetime.now()
    ray.init(ignore_reinit_error=True)
    validation_removed, validation_new = prepare_data(validation)
    print('Time building validation set: {}'.format(datetime.datetime.now()-start))
    validation_removed.to_csv('./' + path + '/validation_removed_tmp.csv')
    validation_new.to_csv('./' + path + '/validation_new.csv')

if (os.path.isfile('./' + path + '/test_removed_tmp.csv') and os.path.isfile('./' + path + '/test_new.csv')):
    print("Test set already exists" )
    test_removed = pd.read_csv('./' + path + '/test_removed_tmp.csv')
    test_new = pd.read_csv('./' + path + '/test_new.csv')                                                                
else:
    start = datetime.datetime.now()
    ray.init(ignore_reinit_error=True)
    test_removed, test_new = prepare_data(test)
    print('Time building test set: {}'.format(datetime.datetime.now()-start))
    test_removed.to_csv('./' + path + '/test_removed_tmp.csv')
    test_new.to_csv('./' + path + '/test_new.csv')

Validation set already exists
Test set already exists


In [13]:
# Return tuple of (user, average rating for user)
@ray.remote
def add_user(dataset, user):
    return (user, np.mean(dataset[dataset["userId"] == user]["rating"]))

In [14]:
# Returns dataset with the average user rating for each user
# Optionally takes in removed when using validation or test data
def user_avg(dataset, using_removed = False, removed = pd.DataFrame()):
    def to_iterator(obj_ids):
        while obj_ids:
            done, obj_ids = ray.wait(obj_ids)
            yield ray.get(done[0])

    # Get the user averages
    dataset["userAvg"] = 0.0
    dat = ray.put(dataset)
    obj_ids = ([add_user.remote(dat, user) for user in dataset.userId.unique()])
    results = []
    for x in tqdm(to_iterator(obj_ids), total = len(obj_ids)):
        results.append(x)
    del dat

    if (using_removed):
        removed["userAvg"] = 0.0
    print("Combining data")
    for result in tqdm(results):
        dataset.loc[dataset["userId"] == result[0], ["userAvg"]] = result[1]
        if (using_removed):
            removed.loc[removed.userId == result[0], ["userAvg"]] = result[1]
    
    return dataset, removed

In [15]:
# Return tuple of (movie, average rating for movie)
@ray.remote
def add_movie(dataset, movie):
    return (movie, np.mean(dataset[dataset["movieId"] == movie]["rating"]))

In [16]:
# Returns dataset with the average movie rating for each movie
def movie_avg(dataset, removed):
    def to_iterator(obj_ids):
        while obj_ids:
            done, obj_ids = ray.wait(obj_ids)
            yield ray.get(done[0])

    # Get the movie averages
    dataset["movieAvg"] = 0.0
    dat = ray.put(dataset)
    obj_ids = ([add_movie.remote(dat, movie) for movie in dataset.movieId.unique()])
    results = []
    for x in tqdm(to_iterator(obj_ids), total = len(obj_ids)):
        results.append(x)
    del dat
    
    removed["movieAvg"] = 0.0
    print("Combining data")
    for result in tqdm(results):
        dataset.loc[dataset["movieId"] == result[0], ["movieAvg"]] = result[1]
        removed.loc[removed.movieId == result[0], ["movieAvg"]] = result[1]
        
    return dataset, removed

In [17]:
# Return datasets with average movie rating and average user rating for each row
def add_avgs(training, validation_new, test_new, validation_removed, test_removed):
    print("\nCalculating training set user averages")
    training, dummy = user_avg(training, False)
    print("\nCalculating validation set user averages")
    validation_new, validation_removed = user_avg(validation_new, True, validation_removed)
    print("\nCalculating test set user averages")
    test_new, test_removed = user_avg(test_new, True, test_removed)
    
    print("\nCalculating validation set movie averages")
    training_validation, validation_removed = movie_avg(pd.concat([training, validation_new]), validation_removed)
    
    print("\nCalculating test set movie averages")
    training_test, test_removed = movie_avg(pd.concat([training, test_new]), test_removed)
    
    return training_validation, training_test, validation_removed, test_removed

In [18]:
if (os.path.isfile('./' + path + '/training_validation.csv') and os.path.isfile('./' + path + '/training_test.csv') and os.path.isfile('./' + path + '/validation_removed.csv') and os.path.isfile('./' + path + '/test_removed.csv')):
    print("Sets already exist" )
    training_validation = pd.read_csv('./' + path + '/training_validation.csv')
    training_test = pd.read_csv('./' + path + '/training_test.csv')
    validation_removed = pd.read_csv('./' + path + '/validation_removed.csv')
    test_removed = pd.read_csv('./' + path + '/test_removed.csv')
else:
    
    # training_validation has all training data as well as the unremoved values in the validation data
    # this will be used to train, and we will use the removed validation data to predict
    
    # training_test has all training data as well as the unremoved values in the test data
    # this will be used to train, and we will use the removed test data to predict
    
    training_validation, training_test, validation_removed, test_removed = add_avgs(training, validation_new, test_new, validation_removed, test_removed)
    training_validation.to_csv('./' + path + '/training_validation.csv')
    training_test.to_csv('./' + path + '/training_test.csv')
    validation_removed.to_csv('./' + path + '/validation_removed.csv')
    test_removed.to_csv('./' + path + '/test_removed.csv')

Sets already exist


# Baselines

In [21]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

## Classification Baselines

In [19]:
y_val = copy.deepcopy(validation_removed['rating']) 
y_val = np.ravel(y_val)
y_val = pd.cut(y_val, bins=[0,3,5], labels=[0,1]) # convert continuous to categorical

y_test = copy.deepcopy(test_removed['rating']) 
y_test = np.ravel(y_test)
y_test = pd.cut(y_test, bins=[0,3,5], labels=[0,1]) # convert continuous to categorical

### Validation Baselines

In [22]:
# Baseline user average
ratings_pred = validation_removed.copy(deep=True)

for user in tqdm(training_validation["userId"].unique()):
    ratings_pred.loc[ratings_pred["userId"] == user, ["rating"]] = np.digitize(training_validation.loc[training_validation.userId == user, "userAvg"].iat[0], bins=[3], right = False)

avg_user_accuracy = accuracy_score(y_val, ratings_pred["rating"])
print("Baseline Validation Accuracy for user avg: %.2f%%" % (avg_user_accuracy * 100))

# Baseline movie average
ratings_pred = validation_removed.copy(deep=True)
training_movies = training_validation.movieId.unique()

for movie in tqdm(validation_removed.movieId.unique()):
    if movie in training_movies:
        ratings_pred.loc[ratings_pred.movieId == movie, ["rating"]] = np.digitize(training_validation.loc[training_validation.movieId == movie, "movieAvg"].iat[0], bins=[3], right = False)
    
avg_movie_accuracy = accuracy_score(y_val, ratings_pred["rating"])
print("Baseline Test Accuracy for movie avg: %.2f%%" % (avg_movie_accuracy * 100))

100%|██████████| 439/439 [00:00<00:00, 1000.53it/s]


Baseline Validation Accuracy for user avg: 71.82%


100%|██████████| 550/550 [00:00<00:00, 1014.03it/s]

Baseline Test Accuracy for movie avg: 72.16%


### Test Baselines

In [23]:
# Baseline user average
ratings_pred = test_removed.copy(deep=True)

for user in tqdm(training_test["userId"].unique()):
    ratings_pred.loc[ratings_pred["userId"] == user, ["rating"]] = np.digitize(training_test.loc[training_test.userId == user, "userAvg"].iat[0], bins=[3], right = False)

avg_user_accuracy = accuracy_score(y_test, ratings_pred["rating"])
print("Baseline Test Accuracy for user avg: %.2f%%" % (avg_user_accuracy * 100))

# Baseline movie average
ratings_pred = test_removed.copy(deep=True)
training_movies = training_test.movieId.unique()

for movie in tqdm(test_removed.movieId.unique()):
    if movie in training_movies:
        ratings_pred.loc[ratings_pred.movieId == movie, ["rating"]] = np.digitize(training_test.loc[training_test.movieId == movie, "movieAvg"].iat[0], bins=[3], right = False)
    
avg_movie_accuracy = accuracy_score(y_test, ratings_pred["rating"])
print("Baseline Test Accuracy for movie avg: %.2f%%" % (avg_movie_accuracy * 100))

100%|██████████| 461/461 [00:00<00:00, 1026.75it/s]


Baseline Test Accuracy for user avg: 69.73%


100%|██████████| 620/620 [00:00<00:00, 1013.82it/s]

Baseline Test Accuracy for movie avg: 70.00%


## Regression Baselines

In [24]:
y_val = copy.deepcopy(validation_removed['rating']) 
y_val = np.ravel(y_val)

y_test = copy.deepcopy(test_removed['rating']) 
y_test = np.ravel(y_test)

### Validation Baselines

In [25]:
# Baseline user average
ratings_pred = validation_removed.copy(deep=True)
for user in tqdm(validation_removed.userId.unique()):
    ratings_pred.loc[ratings_pred["userId"] == user, ["rating"]] = training_validation.loc[training_validation.userId == user, "userAvg"].iat[0]

avg_user_rmse = mean_squared_error(y_val, ratings_pred["rating"], squared = False)
print("Baseline Validation RMSE for user avg:", avg_user_rmse)

# Baseline movie average
ratings_pred = validation_removed.copy(deep=True)
training_movies = training_validation.movieId.unique()

for movie in tqdm(validation_removed.movieId.unique()):
    if movie in training_movies:
        ratings_pred.loc[ratings_pred.movieId == movie, ["rating"]] = training_validation.loc[training_validation.movieId == movie, "movieAvg"].iat[0]
    
avg_movie_rmse = mean_squared_error(y_val, ratings_pred["rating"], squared = False)
print("Baseline Validation RMSE for movie avg:", avg_movie_rmse)

100%|██████████| 88/88 [00:00<00:00, 974.18it/s]


Baseline Validation RMSE for user avg: 0.970671907398001


100%|██████████| 550/550 [00:00<00:00, 1029.39it/s]

Baseline Validation RMSE for movie avg: 0.9279114110631916


### Test Baselines

In [26]:
# Baseline user average
ratings_pred = test_removed.copy(deep=True)
for user in tqdm(training_test["userId"].unique()):
    ratings_pred.loc[ratings_pred["userId"] == user, ["rating"]] = training_test.loc[training_test.userId == user, "userAvg"].iat[0]

avg_user_rmse = mean_squared_error(y_test, ratings_pred["rating"], squared = False)
print("Baseline Test RMSE for user avg:", avg_user_rmse)

# Baseline movie average
ratings_pred = test_removed.copy(deep=True)
training_movies = training_test.movieId.unique()

for movie in tqdm(test_removed.movieId.unique()):
    if movie in training_movies:
        ratings_pred.loc[ratings_pred.movieId == movie, ["rating"]] = training_test.loc[training_test.movieId == movie, "movieAvg"].iat[0]
    
avg_movie_rmse = mean_squared_error(y_test, ratings_pred["rating"], squared = False)
print("Baseline Test RMSE for movie avg:", avg_movie_rmse)

100%|██████████| 461/461 [00:00<00:00, 1024.19it/s]


Baseline Test RMSE for user avg: 0.9830622358141182


100%|██████████| 620/620 [00:00<00:00, 1041.96it/s]

Baseline Test RMSE for movie avg: 1.0358683519569263


# Models

## Decision Tree

#### Classification Approach

In [27]:
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

import seaborn as sns
import matplotlib.pyplot as plt

from mlxtend.evaluate import bootstrap_point632_score

from mlxtend.plotting import plot_decision_regions
from sklearn.tree import plot_tree


In [28]:
use_cols = ['userId', 'movieId', 'userAvg', 'movieAvg', 'timestamp']
use_cols.extend(genres)
x_train = copy.deepcopy(training_validation[use_cols].to_numpy())
y_train = copy.deepcopy(training_validation[['rating']])
y_train = np.ravel(y_train)
y_train = pd.cut(y_train, bins=[0,3,5], labels=[0,1]) # convert continuous to categorical

x_val = copy.deepcopy(validation_removed[use_cols].to_numpy())
y_val = copy.deepcopy(validation_removed['rating']) 
y_val = np.ravel(y_val)
y_val = pd.cut(y_val, bins=[0,3,5], labels=[0,1]) # convert continuous to categorical

In [31]:
tree = DecisionTreeClassifier(random_state=123)
param_grid = {'max_depth':[3,4,5,6,7,8,9,10,15,20,None], 
              'criterion':['gini','entropy']} 

gs = GridSearchCV(estimator=tree,
                  param_grid=param_grid,
                  scoring='accuracy',
                  #scoring='neg_root_mean_squared_error',
                  cv=10,
                  n_jobs=-1)

gs.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=DecisionTreeClassifier(random_state=123),
             n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [3, 4, 5, 6, 7, 8, 9, 10, 15, 20, None]},
             scoring='accuracy')

In [32]:
print('Best Params: %s' % gs.best_params_)
print('Best Validation Accuracy: %.2f%%' % (gs.best_score_*100))

Best Params: {'criterion': 'gini', 'max_depth': 4}
Best Validation Accuracy: 73.40%


In [33]:
# Fit best model on whole dataset and predict validation data
tree = DecisionTreeClassifier(**gs.best_params_, random_state=123)
tree.fit(x_train, y_train)

ratings_pred = tree.predict(x_val)
accuracy = accuracy_score(y_val, ratings_pred)
print('Validation Accuracy: %.2f%%' % (accuracy*100))

Validation Accuracy: 76.02%


In [34]:
# Fit best model on whole dataset and predict test data
x_train = copy.deepcopy(training_test[use_cols].to_numpy())

y_train = copy.deepcopy(training_test[['rating']])
y_train = np.ravel(y_train)
y_train = pd.cut(y_train, bins=[0,3,5], labels=[0,1])

x_test = copy.deepcopy(test_removed[use_cols].to_numpy())
y_test = copy.deepcopy(test_removed['rating']) 

y_test = np.ravel(y_test)
y_test = pd.cut(y_test, bins=[0,3,5], labels=[0,1]) # convert continuous to categorical

tree = DecisionTreeClassifier(**gs.best_params_, random_state=123)
tree.fit(x_train, y_train)

ratings_pred = tree.predict(x_test)
accuracy = accuracy_score(y_test, ratings_pred)
print('Test Accuracy: %.2f%%' % (accuracy*100))

Test Accuracy: 75.91%


#### Regression Approach

In [37]:
use_cols = ['userId', 'movieId', 'userAvg', 'movieAvg', 'timestamp']
use_cols.extend(genres)
x_train = copy.deepcopy(training_validation[use_cols].to_numpy())
y_train = copy.deepcopy(training_validation[['rating']])
y_train = np.ravel(y_train)

x_val = copy.deepcopy(validation_removed[use_cols].to_numpy())
y_val = copy.deepcopy(validation_removed['rating']) 
y_val = np.ravel(y_val)

In [35]:
tree = DecisionTreeRegressor(random_state=123)
param_grid = {'max_depth':[3,4,5,6,7,8,9,10,15,20,None]} 

gs = GridSearchCV(estimator=tree,
                  param_grid=param_grid,
                  #scoring='neg_mean_absolute_percentage_error',
                  scoring='neg_root_mean_squared_error',
                  cv=10,
                  n_jobs=-1)

gs.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=DecisionTreeRegressor(random_state=123),
             n_jobs=-1,
             param_grid={'max_depth': [3, 4, 5, 6, 7, 8, 9, 10, 15, 20, None]},
             scoring='neg_root_mean_squared_error')

In [38]:
print('Best Params:',  gs.best_params_)
print('Best Validation RMSE:', -1 * gs.best_score_)

Best Params: {'max_depth': 6}
Best Validation RMSE: 0.8342672177840438


In [39]:
# Fit best model on whole dataset and predict validation data
tree = DecisionTreeRegressor(**gs.best_params_, random_state=123)
tree.fit(x_train, y_train)

ratings_pred = tree.predict(x_val)
rmse = mean_squared_error(y_val, ratings_pred, squared = False)
print('Validation RMSE:', rmse)

Validation RMSE: 0.9203161079642824


In [40]:
# Fit best model on whole dataset and predict test data
x_train = copy.deepcopy(training_test[use_cols].to_numpy())
y_train = copy.deepcopy(training_test[['rating']])
y_train = np.ravel(y_train)

x_test = copy.deepcopy(test_removed[use_cols].to_numpy())
y_test = copy.deepcopy(test_removed['rating']) 
y_test = np.ravel(y_test)

tree = DecisionTreeRegressor(**gs.best_params_, random_state=123)
tree.fit(x_train, y_train)

ratings_pred = tree.predict(x_test)
rmse = mean_squared_error(y_test, ratings_pred, squared = False)
print('Test RMSE:', rmse)

Test RMSE: 0.903037475964926


## XGBoost

In [41]:
from xgboost import XGBClassifier
from xgboost import XGBRegressor

### Classification Approach

In [42]:
# Fit best model on whole dataset and predict validation data
use_cols = ['userId', 'movieId', 'userAvg', 'movieAvg', 'timestamp']
use_cols.extend(genres)
x_train = copy.deepcopy(training_validation[use_cols].to_numpy())

y_train = copy.deepcopy(training_validation[['rating']])
y_train = np.ravel(y_train)
y_train = pd.cut(y_train, bins=[0,3,5], labels=[0,1]) # convert continuous to categorical

x_val = copy.deepcopy(validation_removed[use_cols].to_numpy())

y_val = copy.deepcopy(validation_removed['rating']) 
y_val = np.ravel(y_val)
y_val = pd.cut(y_val, bins=[0,3,5], labels=[0,1]) # convert continuous to categorical

clf = XGBClassifier(eval_metric = 'logloss', eta=0.1, max_depth=5, min_child_weight = 4, n_estimators = 200, subsample = 0.70, n_jobs=-1, random_state=1, use_label_encoder = False)
clf.fit(x_train, y_train)

ratings_pred = clf.predict(x_val)
accuracy = accuracy_score(y_val, ratings_pred)
print('Validation Accuracy: %.2f%%' % (accuracy*100))

Validation Accuracy: 78.07%


In [43]:
# Fit best model on whole dataset and predict test data
x_train = copy.deepcopy(training_test[use_cols].to_numpy())

y_train = copy.deepcopy(training_test[['rating']])
y_train = np.ravel(y_train)
y_train = pd.cut(y_train, bins=[0,3,5], labels=[0,1])

x_test = copy.deepcopy(test_removed[use_cols].to_numpy())
y_test = copy.deepcopy(test_removed['rating']) 

y_test = np.ravel(y_test)
y_test = pd.cut(y_test, bins=[0,3,5], labels=[0,1]) # convert continuous to categorical

clf = XGBClassifier(eval_metric = 'logloss', eta=0.1, max_depth=5, min_child_weight = 4, n_estimators = 200, subsample = 0.70, n_jobs=-1, random_state=1, use_label_encoder = False)
clf.fit(x_train, y_train)

ratings_pred = clf.predict(x_test)
accuracy = accuracy_score(y_test, ratings_pred)
print('Test Accuracy: %.2f%%' % (accuracy*100))

Test Accuracy: 77.45%


### Regression Approach

In [44]:
# Fit best model on whole dataset and predict validation data
use_cols = ['userId', 'movieId', 'userAvg', 'movieAvg', 'timestamp']
use_cols.extend(genres)
x_train = copy.deepcopy(training_validation[use_cols].to_numpy())

y_train = copy.deepcopy(training_validation[['rating']])
y_train = np.ravel(y_train)

x_val = copy.deepcopy(validation_removed[use_cols].to_numpy())
y_val = copy.deepcopy(validation_removed['rating']) 
y_val = np.ravel(y_val)

clf = XGBRegressor(eval_metric = 'logloss', eta=0.1, max_depth=5, min_child_weight = 4, n_estimators = 200, subsample = 0.70, n_jobs=-1, random_state=1, use_label_encoder = False)
clf.fit(x_train, y_train)

ratings_pred = clf.predict(x_val)
rmse = mean_squared_error(y_val, ratings_pred, squared = False)
print('Validation RMSE:', rmse)

Validation RMSE: 0.9038550284257149


In [45]:
# Fit best model on whole dataset and predict validation data
x_train = copy.deepcopy(training_test[use_cols].to_numpy())
y_train = copy.deepcopy(training_test[['rating']])
y_train = np.ravel(y_train)

x_test = copy.deepcopy(test_removed[use_cols].to_numpy())
y_test = copy.deepcopy(test_removed['rating']) 
y_test = np.ravel(y_test)

clf = XGBRegressor(eval_metric = 'logloss', eta=0.1, max_depth=5, min_child_weight = 4, n_estimators = 200, subsample = 0.70, n_jobs=-1, random_state=1, use_label_encoder = False)
clf.fit(x_train, y_train)

ratings_pred = clf.predict(x_test)
rmse = mean_squared_error(y_test, ratings_pred, squared = False)
print('Test RMSE:', rmse)

Test RMSE: 0.8682323577760647


#### .632+ Bootstrap

In [ ]:
"""
scores = bootstrap_point632_score(gs, x_train, y_train, n_splits=50, 
                                  method='.632+', random_seed=123) 
acc = np.mean(scores)
print('Accuracy: %.2f%%' % (100*acc))

# 95% confidence interval
lower = np.percentile(scores,2.5)*100
upper = np.percentile(scores,97.5)*100
print('95%% Confidence interval: [%.2f, %.2f]' % (lower, upper))
"""

# Plots

In [ ]:
best_model = DecisionTreeClassifier(**gs.best_params_, random_state=123)
best_model.fit(x_train,y_train)

In [ ]:
plt.figure(figsize=(10, 7))
plot_tree(best_model, 
          filled=True, 
          rounded=True,
          class_names=['0', 
                       '1'],
          feature_names=training_test[use_cols].columns) 

plt.show()

In [ ]:
tree_estimator = gs.best_estimator_
tree_estimator

In [ ]:
tree_estimator.feature_importances_

In [ ]:
feature_importance_values = tree_estimator.feature_importances_
feature_importances = pd.Series(feature_importance_values, index=training_test[use_cols].columns) # x train's column values
feature_top20 = feature_importances.sort_values(ascending=False)[:20]

plt.figure(figsize=[8, 6])
plt.title('Feature Importances Top 20')
sns.barplot(x=feature_top20, y=feature_top20.index)
plt.show()

- movieAverage was the most important feature in the prediction model
    - movie's quality/social standard in general affects individuals' rate 
- userAverage was the second > so each person's rating standard/rating tendancy affects actual rate
    - someone could send out 0 for dislike and 3 or like but someone else could send 5 for all movies
    
- timestamp represents seconds that have elapsed since the Unix epoch 00:00:00 UTC on midnight UTC of Janurary 1, 1970
    - https://en.wikipedia.org/wiki/Unix_time
    - Thus when the movie was watched also affects the rate
    
- genres in the other hand doesn't significantly affect the rate 

In [ ]:
# plot_decision_regions(np.asarray(copy.deepcopy(training_test[plot_cols])).astype('float32'), np.array(copy.deepcopy(y_train)), gs)

# plt.tight_layout()
# plt.show()